#  Few-shot 프롬프팅

---

# 환경 설정 및 준비

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [ ]:
import os
from glob import glob

from pprint import pprint
import json

`(3) LLM 설정`

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.3,
    top_p=0.9
)

# **Zero-shot** 프롬프팅

- **Zero-shot 프롬프팅**은 예시 없이 AI가 즉시 작업을 수행하는 기법입니다

- 명확한 **지시사항**만으로 원하는 결과를 얻을 수 있어 **사용이 간단**합니다

- 단순하고 직관적인 작업에 적합한 프롬프팅 방식이지만, 작업의 **복잡도에 따라 선택적 사용**이 필요합니다

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Zero-shot 프롬프트 템플릿 생성
zero_shot_prompt = PromptTemplate(
    input_variables=["topic"],
    template="다음 시장에서 삼성전자의 경쟁업체를 설명해주세요: {topic}"
)

# 체인 생성
chain = zero_shot_prompt | llm | StrOutputParser()

# Zero-shot 실행
topic = "인공지능 반도체"
zero_shot_result = chain.invoke(input={"topic": topic}) 

print(zero_shot_result)

In [ ]:
# Zero-shot 프롬프팅 - 컨텍스트(Context) 제공 
zero_shot_prompt = PromptTemplate(
    input_variables=["context", "topic"],
    template="""{topic} 시장에서 삼성전자의 경쟁업체를 설명해주세요. 
    반드시 다음 제시된 뉴스에 근거해서 답변하세요:

    [뉴스]
    {context}
    
    [답변]
    """
)


# 체인 생성
chain = zero_shot_prompt | llm | StrOutputParser()

# Zero-shot 실행
context = """삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 
이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 
세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.
"""

topic = "인공지능 반도체"
zero_shot_result = chain.invoke(input={"context": context, "topic": topic})

print(zero_shot_result)

# **One-shot** 프롬프팅

- **One-shot 프롬프팅**은 하나의 예시를 통해 AI가 작업 패턴을 학습하는 기법입니다

- **Zero-shot** 방식보다 더 나은 성능을 제공하며, **형식화된 작업**에 특히 효과적입니다

- 단일 예시로 **품질 향상**이 가능하나, 해당 예시에 **과의존**할 수 있는 한계가 있습니다

In [ ]:
### One-shot 프롬프트 템플릿 생성
# 1. Zero-shot 프롬프트 템플릿에 예시(example)를 포함하도록 수정
# 2. input_variables에 example_topic과 example_response 추가

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

one_shot_prompt = PromptTemplate(
    input_variables=["example_topic", "example_response", "topic"],
    template="""다음은 특정 시장에서 삼성전자의 경쟁업체를 설명하는 예시이다:

시장: {example_topic}
경쟁업체: {example_response}

이제 다음 시장에서 삼성전자의 경쟁업체를 설명해주세요:
시장: {topic}"""
)

# Example 데이터 설정
example_topic = "스마트폰"
example_response = """애플: 프리미엄 시장에서 주요 경쟁사로, iPhone 시리즈로 경쟁
샤오미: 중저가 시장에서 강세를 보이며 글로벌 시장 점유율 확대
구글: Pixel 시리즈로 프리미엄 시장 진출, AI 기능 강조"""

`(1) PromptTemplate 그대로 사용`

In [ ]:
# one_shot_prompt 적용한 체인 생성
chain = one_shot_prompt | llm | StrOutputParser()

# One-shot 실행
topic = "인공지능 반도체"
one_shot_result = chain.invoke(
    input={
        "example_topic": example_topic,
        "example_response": example_response,
        "topic": topic
    }
)

print(f"one_shot_result:")
print(one_shot_result)

`(2) partial 메소드 사용`

In [ ]:
# Example 데이터를 반영한 부분 프롬프트 출력 
partial_prompt = one_shot_prompt.partial(
    example_topic=example_topic,
    example_response=example_response,
)

print(f"partial_prompt:")
print(partial_prompt)

# 체인 생성
chain = partial_prompt | llm | StrOutputParser()

# One-shot 실행
topic = "인공지능 반도체"
one_shot_result = chain.invoke(input={"topic": topic})

print(f"one_shot_result:")
print(one_shot_result)

# **Few-shot** 프롬프팅

- **Few-shot 프롬프팅**은 AI 모델에게 2-5개의 예시를 제공하여 학습시키는 방법입니다

- 이 방식은 **Zero-shot**이나 **One-shot** 프롬프팅보다 더 우수한 성능을 보여주며, 복잡한 작업에서 특히 효과적입니다

- Few-shot 프롬프팅은 높은 성능을 제공하지만, 긴 프롬프트로 인한 **비용 증가**를 고려해야 합니다

`(1) PromptTemplate 사용`

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Few-shot 프롬프트 템플릿 생성 
few_shot_prompt = PromptTemplate(
   input_variables=["examples", "topic"],
   template="""다음은 여러 시장에서 삼성전자의 경쟁업체를 설명하는 예시들이다:

{examples}

이제 다음 시장에서 삼성전자의 경쟁업체를 설명해주세요:
시장: {topic}"""
)

# Example 데이터 준비
examples = """
시장: 스마트폰
경쟁업체: 
- 애플(미국): 프리미엄 시장 주도, iPhone으로 경쟁
- 샤오미(중국): 중저가 시장 강세, 글로벌 확장중
- 구글(미국): Pixel로 AI 기능 강조

시장: TV
경쟁업체:
- LG전자(한국): OLED 기술 경쟁
- Sony(일본): 프리미엄 시장 경쟁
- TCL(중국): 중저가 시장 공략
"""

# 체인 생성 및 실행
chain = few_shot_prompt | llm | StrOutputParser()
result = chain.invoke({
   "examples": examples,
   "topic": "인공지능 반도체"
})

print(result)

`(2) partial 메소드 사용`

In [ ]:
# partial 메서드를 사용하여 Few-shot 프롬프트 템플릿 생성
partial_prompt = few_shot_prompt.partial(
    examples=examples
)

print(f"partial_prompt:")
print(partial_prompt)

In [ ]:
# 체인 생성
chain = partial_prompt | llm | StrOutputParser()

# Few-shot 실행
topic = "인공지능 반도체"
few_shot_result = chain.invoke(input={"topic": topic})

print(f"few_shot_result:")
print(few_shot_result)

`(3) FewShotChatMessagePromptTemplate 사용`

* FewShotChatMessagePromptTemplate는 LangChain에서 제공하는 템플릿으로, **미리 정의된 고정된 예제들(Fixed Examples)** 을 프롬프트에 포함시켜 모델이 일관된 형식과 품질의 응답을 생성할 수 있도록 돕습니다.

* 이 방식은 특히 특정 형식이나 구조를 가진 출력이 필요한 경우(예: JSON 형식, 특정 분석 리포트 형식 등) 매우 유용하며, 예제들이 고정되어 있어 결과의 일관성을 보장할 수 있습니다.

* 단, 고정된 예제를 사용하기 때문에 상황에 따라 유연하게 대응하기 어려울 수 있으며, 모든 케이스를 커버하기 위해서는 신중한 예제 선택이 필요합니다.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from textwrap import dedent # text의 모든 줄에서 같은 선행 공백을 제거하는 함수

# 예시 데이터 정의 : 뉴스 텍스트(input) + 키워드 추출 결과 (output)
examples = [
    {
        "input": dedent("""
                        정부는 의과대학 입학 정원을 2000명 증가시킬 계획의 세부사항을 이달 20일에 공개할 예정이다. 
                        지역별 의료 서비스 향상과 소규모 의과대학의 발전을 목표로, 지역 중심의 국립대학 및 소형 의과대학의 
                        입학 정원이 최소한 두 배 가량 확대될 것으로 보인다.
                        """),
        "output": "의대 | 정원 | 확대"
    },
    {
        "input": dedent("""
                        세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 
                        전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.
                        """),
        "output": "세계보건기구 | 건강위기 | 국제"
    }
]

# 각 예시를 포맷팅할 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("assistant", "{output}")
])

# Few-shot 프롬프트 템플릿 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,      # 예시 포맷팅 템플릿
    examples=examples                   # 예시 데이터 리스트 -> 예시 포맷팅 템플릿에 적용
)

pprint(few_shot_prompt.invoke({}).to_messages())

In [ ]:
# 최종 프롬프트 템플릿 생성
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 뉴스 텍스트에서 핵심 키워드 3개를 추출하는 전문가입니다."),
    few_shot_prompt,
    ("human", "{input}")
])

# 프롬프트 템플릿 출력
pprint(final_prompt.invoke({"input": "뉴스 기사입니다"}).to_messages())

In [ ]:
# 키워드 추출 체인 생성
chain = final_prompt | llm | StrOutputParser()

# 키워드 추출 체인 실행
result = chain.invoke({
    "input": dedent("""삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 
                    이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 
                    세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.""")
})

print(result)

`(4) Dynamic Few-Shot Prompting`

* **Dynamic Few-Shot Prompting**은 상황에 따라 적절한 예시를 동적으로 선택하여 사용하는 고급 프롬프팅 기법으로, **BaseExampleSelector**를 통해 입력값과 가장 연관성이 높은 예시들을 자동으로 선별합니다.

* 대표적으로 **SemanticSimilarityExampleSelector**는 의미적 유사도를 기반으로 예시를 선택하며, 이를 통해 주어진 입력 상황에 가장 적합한 예시들만을 효율적으로 활용할 수 있습니다.

* **example_prompt**를 통해 선택된 예시들을 AI 시스템이 이해하기 쉬운 형태(예: human-AI 대화 , human-function call)로 변환하여 더 효과적인 학습과 응답 생성이 가능하게 합니다.


- **장점**

    - 상황에 맞는 가장 연관성 높은 예시만을 선택적으로 활용할 수 있다
    - 프롬프트의 길이를 효율적으로 관리할 수 있다
    - 응답의 일관성과 품질을 향상시킬 수 있다

In [ ]:
from langchain_ollama import OllamaEmbeddings 
from langchain_core.vectorstores import InMemoryVectorStore # type: ignore

# 고객 문의 유형별 응대 예시를 준비
examples = [
    {
        "input": "환불 절차가 어떻게 되나요?",
        "output": "환불 절차는 다음과 같습니다:\n1. 구매내역에서 환불을 신청해주세요\n2. 반품 상품을 발송해주세요\n3. 상품 검수 후 3-5일 내 환불이 완료됩니다"
    },
    {
        "input": "배송이 늦어지고 있어요", 
        "output": "불편을 드려 죄송합니다. 주문번호를 알려주시면 배송 상태를 즉시 확인해드리겠습니다."
    },
    {
        "input": "옷 사이즈가 안 맞아요",
        "output": "사이즈 교환은 무료로 진행됩니다. 교환 신청 후 동일 상품의 다른 사이즈로 발송해드리겠습니다."
    },
    {
        "input": "제품이 불량이에요",
        "output": "불편을 드려 대단히 죄송합니다. 불량 부분 사진과 함께 1:1 문의에 접수해주시면 빠르게 처리해드리겠습니다."
    }
]

# 예시 데이터를 벡터화할 텍스트로 변환
to_vectorize = [" ".join(example.values()) for example in examples]

# Ollama 임베딩 모델 생성
embeddings = OllamaEmbeddings(model="bge-m3")

# 벡터 스토어 생성
vector_store = InMemoryVectorStore.from_texts(
    to_vectorize,    # 벡터화할 텍스트 리스트
    embeddings,      # 임베딩 모델
    metadatas=examples    # 메타데이터: 예시 데이터
    )

# VectorStore에 저장된 Document 개수 확인
print(f"VectorStore에 저장된 Document 개수: {len(vector_store.store.keys())}")

In [ ]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

# 유사한 2개의 예시를 선택하는 selector 생성
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vector_store,
    k=2
)

# 선택된 예시 확인
selected_examples = example_selector.select_examples({"input":"상품이 파손되어 왔어요"})
pprint(selected_examples)

In [ ]:
# 챗봇 프롬프트 템플릿 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["input"],
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages([
        ("human", "{input}"),
        ("assistant", "{output}")
    ])
)

pprint(few_shot_prompt.invoke({"input": "상품이 파손되어 왔어요"}).to_messages())

In [ ]:
# 최종 프롬프트 생성 
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 친절하고 전문적인 고객 서비스 담당자입니다."),
    few_shot_prompt,
    ("human", "{input}")
])

pprint(final_prompt.invoke({"input": "상품이 파손되어 왔어요"}).to_messages())

In [ ]:
# 챗봇 체인 생성
chain = final_prompt | llm | StrOutputParser()

# 체인 실행
response = chain.invoke({
    "input": "상품이 파손되어 왔어요"
})

pprint(response)